# Staking Slashing Scenarios

## 1. Resumption from Inactivity

## Summary

### Objective 
The Resumption from Inactivity scenario evaluates the network's ability to successfully resume activities after a portion of the network has been slashed due to inactivity. While mass slashings of consensus participants should only happen in extreme cases, the current slashing model theoretically allows validators to slash each other for arbitrary reasons. Even though it is too early to simulate specific attacks, it is necessary to further understand the effects of such events - even though the underlying reasons that lead to such an event might not be clear yet. If a high share of validators become inactive due to slashing, the network can become inactive. 
This scenario assesses how successful the network is at resuming activities after a proportion of the network has been slashed due to inactivity.

### Experimental Setup

#### Testing Variables: 
:dart: we should check the below variables 

##### Environmental:
- Simulating time series for validator events

##### Protocol: 
- Assess the network's ability to resume finalization after slashing-induced inactivity. This scenario tests various parameter choices related to validator behavior and slashing mechanisms. Key parameters include:  
    i) Probability of drawing a slashable offense for each validator  
    ii) Probability of specific offenses occurring  
    iii) Conditions leading to validator removal from the network

#### Simulation input/output per Monte Carlo run:

##### Input:
- Time series of events drawn from a set of events that include slashable offenses. These events are used to assess the impact on network activity and resumption.

##### Output:  
- Time series of epoch finalization events on L1, capturing the periods of network activity and resumption from inactivity.
- Monte Carlo runs ($M$ runs indexed by $m$): 
    For each parameter constellation of interest, a realization from a set of events is drawn for every validator at every block. Some (perhaps all) of the events in the set are slashable offenses, and some contribute to network inactivity when a validator is 1) offline, as evidenced by an event realization, or 2) removed by the protocol, after accumulating too many offenses. The realization probability may be coarsely specified as a fixed probability of drawing a slashable offense for each validator, and then another probability of which offense has occurred. A non-slashable event is ignored.

#### Sweep Parameters:
Refer to the [spreadsheet](https://docs.google.com/spreadsheets/d/1EbW4sEYWb7iCjOYfgivNsLxe1O6ZFYwFSsRQjfiPLTM/edit?gid=955157985#gid=955157985&range=A1) for detailed parameter configurations related to staking and slashing mechanisms.

:dart: Fill in :dart:

##### Control:

##### Environmental:

#### Simulation Behavior: 

#### Threshold Inequalities:

#### Metrics:
1. **Percentage of Finalization Resumption from Inactivity:**
    
    - Defined as the fraction of Monte Carlo runs for which finalization has fully resumed by the end of the run, where validators have been inactive due to slashing.
    - **Interpretation:** A resilient network resumes finalization after slashing events have caused a number of validators to become inactive. This is represented by a high percentage of runs exhibiting full finalization resumption by the end of the simulation run.

In [ ]:
#### Computational Complexity:
1. Total number of parameter constellations: :dart: 
2. Total number of Monte Carlo runs per constellation: :dart:
3. Total number of experiments per adaptive grid: :dart:
4. Number of adaptive grid searches: :dart:
5. Total number of parameter constellations evaluated: :dart:

### Adaptive Grid Results 

:::info
See [main doc](https://hackmd.io/@blockscience/B1QKItvEye) for copyable explanations (or referenceable)
:::

- [ ] :dart: Write descriptive interpretation over the plot below :dart: 
- [ ] :dart: Generate an analogue plot below for the results :dart: 

### Protocol Parameter Recommendations 

:::info
See [main doc](https://hackmd.io/@blockscience/B1QKItvEye) for copyable explanations (or referenceable)
:::

- [ ] :dart: Write descriptive interpretation over table :dart: 
- [ ] :dart: Generate an analogue table below for the results :dart: 

### Decision Tree and Parameter Importance

:::info
See [main doc](https://hackmd.io/@blockscience/B1QKItvEye) for copyable explanations (or referenceable)
:::
- [ ] :dart: Write descriptive interpretation over the plot below :dart: 
- [ ] :dart: Generate an analogue plot below for the results :dart:

### Commentary on Results:

:dart: Write anything that comes up to mind based on the interpretation over all the results :dart: 

## Conclusion

- [ ] 🔫 Jakob owns after Danilo wrote commentary on results 🔫

## Appendix:
### Parameter Impact on Metrics: 

## 2. Validator Ejection

### Objective 
The Validator Ejection scenario evaluates the potential risks of creating periods of downtime due to slashing behavior. 
While mass slashings of consensus participants should only happen in extreme cases, the current slashing model theoretically allows validators to slash each other for arbitrary reasons. Even though it is too early to simulate specific attacks, it is necessary to further understand the effects of such events - even though the underlying reasons that lead to such an event might not be clear yet. If a high share of validators are slashed - and ejected from the active validator set (aka "jailed", or "exited") - finalization of the L2 chain might stop. Understanding what happens after such events can help further designing and refining the slashing conditions and magnitudes, as well as fail-safes that help return to active finalization sooner. 
This scenario specifically examines the likelihood of creating periods without finalization after validators are ejected from the network due to slashable offenses.

### Experimental Setup

#### Testing Variables: 
:dart: we should check variables below 

##### Environmental:
- Simulating time series for validator slashing events leading to ejection
- Probability of drawing a slashable offense for each validator


##### Protocol: 
- Assess the impact of validator ejection on network activity. This scenario tests various parameter choices related to validator behavior and slashing mechanisms. Key parameters include:  

#### Simulation input/output per Monte Carlo run:

##### Input:
- Time series of events, drawn from a set of events that include slashable offenses

##### Output:  
- Time series of epoch finalization events on L1, capturing the periods of network activity and resumption from inactivity.
- Monte Carlo ($M$ runs indexed by $m$):
    For each parameter constellation of interest, a realization from a set of events is drawn for every validator at every block. Some (perhaps all) of the events in the set are slashable offenses, and some contribute to network inactivity when a validator is 1) offline, as evidenced by an event realization, or 2) removed by the protocol, after accumulating too many offenses. The realization probability may be coarsely specified as a fixed probability of drawing a slashable offense for each validator, and then another probability of which offense has occurred. A non-slashable event is ignored.

#### Sweep Parameters:
Refer to the [spreadsheet](https://docs.google.com/spreadsheets/d/1EbW4sEYWb7iCjOYfgivNsLxe1O6ZFYwFSsRQjfiPLTM/edit?gid=955157985#gid=955157985&range=A1) for detailed parameter configurations related to staking and slashing mechanisms.

:dart: Fill in :dart:

##### Control:

##### Environmental:

#### Simulation Behavior: 

#### Threshold Inequalities:

#### Metrics:
1. **Average Percentage of Inactivity:**    
- Defined as the average over all Monte Carlo runs of the fraction of time the network was inactive due to forced validator exit from slashing.
- Interpretation: A high percentage of inactivity may exceed a pre-defined upper bound (defined from stakeholder requirements), and indicate that although slashing events are applied per validator, some parameter constellations may be more likely to create "clusters" of validator inactivity.

#### Computational Complexity:
1. Total number of parameter constellations: :dart: 
2. Total number of Monte Carlo runs per constellation: :dart:
3. Total number of experiments per adaptive grid: :dart:
4. Number of adaptive grid searches: :dart:
5. Total number of parameter constellations evaluated: :dart:

### Adaptive Grid Results 

:::info
See [main doc](https://hackmd.io/@blockscience/B1QKItvEye) for copyable explanations (or referenceable)
:::

- [ ] :dart: Write descriptive interpretation over the plot below :dart: 
- [ ] :dart: Generate an analogue plot below for the results :dart: 

### Protocol Parameter Recommendations 

:::info
See [main doc](https://hackmd.io/@blockscience/B1QKItvEye) for copyable explanations (or referenceable)
:::

- [ ] :dart: Write descriptive interpretation over table :dart: 
- [ ] :dart: Generate an analogue table below for the results :dart: 

### Decision Tree and Parameter Importance

:::info
See [main doc](https://hackmd.io/@blockscience/B1QKItvEye) for copyable explanations (or referenceable)
:::
- [ ] :dart: Write descriptive interpretation over the plot below :dart: 
- [ ] :dart: Generate an analogue plot below for the results :dart:

### Commentary on Results:

:dart: Write anything that comes up to mind based on the interpretation over all the results :dart: 

## Conclusion

- [ ] 🔫 Jakob owns after Danilo wrote commentary on results 🔫

## Appendix:
### Parameter Impact on Metrics: 